In [74]:
import pandas as pd
import numpy as np
import plotly as py
import uuid

In [75]:
df = pd.read_excel('/Library/Python_local/TallPallApp/MAI/Doku Langenbach 29-04-24.xlsx', sheet_name='LT22 Export', header=0)

In [76]:
from Data_Class.MMSQL_connection import read_Table, save_Table
# filter df by Queue GITPD 
dfGITPD = df[df['Queue'] == 'GITPD']
dfMasterdata = read_Table('data_materialmaster-MaterialMasterUnitOfMeasures')
dfPAMSMasterdata = read_Table('PAMS_TALL_MASTERDATA')
dfMasterdata


,UnitOfMeasure,MaterialNumber,SnapshotId,NumeratorToBaseUnitOfMeasure,DenominatorToBaseUnitOfMeasure,InternationalArticleNumber,InternationalArticleNumberCategory,Length,Width,Height,UnitOfDimension,Volume,UnitOfVolume,GrossWeight,UnitOfWeight,LowerUnitPackHierarchy,MaxStackFactor
0,023,000000000040023570,00000000-0000-0000-0000-000000000000,19350,1,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
1,023,000000000040056187,00000000-0000-0000-0000-000000000000,2,1,None,None,0.0,0.0,0.0,None,0.0,None,3.26,KGM,None,0
2,033,000000000040036795,00000000-0000-0000-0000-000000000000,25,1,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
3,060,000000000040037859,00000000-0000-0000-0000-000000000000,480,1,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
4,060,000000000040039618,00000000-0000-0000-0000-000000000000,1512,100,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200367,TU,000000000041808704,00000000-0000-0000-0000-000000000000,3,10,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
200368,TU,000000000041812012,00000000-0000-0000-0000-000000000000,1,4,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
200369,TU,000000000041816847,00000000-0000-0000-0000-000000000000,33,100,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0
200370,TU,000000000041820126,00000000-0000-0000-0000-000000000000,1,1,None,None,0.0,0.0,0.0,None,0.0,None,0.00,None,None,0


In [78]:
# def cal_Order_Qty_and_MasterData_SAVE_TO_DATABASE(dfMasterdata: pd.DataFrame, df: pd.DataFrame):
#     '''This function calculates the Order Quantity for Case and PAL and merges the Masterdata to the df
#     :return: save to SQL Table = Tall_Pall_Orders'''

dfMasterdata['MaterialNumber'] = dfMasterdata['MaterialNumber'].str.replace('0000000000', '')

df['Material'] = df['Material'].astype(str)
df_merged_Case = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'CS'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_Case['MM_TH_Pro_Karton'] = df_merged_Case.NumeratorToBaseUnitOfMeasure / df_merged_Case.DenominatorToBaseUnitOfMeasure
df['MM_TH_Pro_Karton'] = df_merged_Case['MM_TH_Pro_Karton']
df['height_Case'] = df_merged_Case['Height']
df['length_Case'] = df_merged_Case['Length']
df['width_Case'] = df_merged_Case['Width']

df_merged_pal = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'D97'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_pal['MM_TH_Pro_Palette'] = df_merged_pal.NumeratorToBaseUnitOfMeasure / df_merged_pal.DenominatorToBaseUnitOfMeasure
df['MM_TH_Pro_Palette'] = df_merged_pal['MM_TH_Pro_Palette']
df['height_PAL'] = df_merged_pal['Height']
df['length_PAL'] = df_merged_pal['Length']
df['width_PAL'] = df_merged_pal['Width']

df_merged_PD = pd.merge(df, dfMasterdata[dfMasterdata['UnitOfMeasure'] == 'PD'], left_on='Material', right_on='MaterialNumber', how='left')
df_merged_PD['Case_on_Layer'] = df_merged_PD.NumeratorToBaseUnitOfMeasure / df_merged_PD.DenominatorToBaseUnitOfMeasure
df['Case_on_Layer'] = df_merged_PD['Case_on_Layer']

# ########## UNTERBAU  ##########
dfPAMSMasterdata['SKU'] = dfPAMSMasterdata['SKU'].astype(str)
df = pd.merge(df, dfPAMSMasterdata, left_on='Material', right_on='SKU', how='left')
nan_indices = df[df['AUFPACKEN_JA_NEIN'].isna()].index
df.loc[nan_indices, 'GEEIGNET_UNTERBAU'] = 'FALSE'
df.loc[nan_indices, 'AUFPACKEN_JA_NEIN'] = 'FALSE'
df


,Material,Source Storage Bin,Source target qty,Alternative Unit of Measure,Queue,Batch,Transfer Order Number,Dest.Storage Bin,Filler,Filler.1,...,length_PAL,width_PAL,Case_on_Layer,SKU,AUFPACKEN_JA_NEIN,GEEIGNET_UNTERBAU,FREITEXT_1,FREITEXT_2,IMG_1,IMG_2
0,10192165,R-12-02-12,210.00,TH,GITPD,16491912.0,1879454,3202656508,NaN,NaN,...,120.0,80.0,70.00,10192165,True,True,None,None,None,None
1,10198439,R-11-01-03,8.61,KG,CASE,74102024.0,1879455,3202656509,NaN,NaN,...,120.0,80.0,8.61,10198439,True,True,None,None,None,None
2,10185733,R-04-01-07,124.20,KG,GITPD,55142024.0,1879456,3202656509,NaN,NaN,...,120.0,80.0,37.80,10185733,True,False,None,None,None,None
3,10185733,R-04-01-08,124.20,KG,GITPD,55142024.0,1879457,3202656509,NaN,NaN,...,120.0,80.0,37.80,10185733,True,False,None,None,None,None
4,10179241,R-11-03-14,124.20,KG,GITPD,55152024.0,1879458,3202656509,NaN,NaN,...,120.0,80.0,37.80,10179241,True,False,None,None,None,None
5,10198491,G7E12,37.60,KG,GITPD,74142024.0,1879459,3202656509,NaN,NaN,...,120.0,80.0,4.70,10198491,True,True,None,None,None,None
6,10198491,G7E12,37.60,KG,GITPD,74142024.0,1879460,3202656509,NaN,NaN,...,120.0,80.0,4.70,10198491,True,True,None,None,None,None
7,10198491,G7B03,37.60,KG,GITPD,74152024.0,1879601,3202656509,NaN,NaN,...,120.0,80.0,4.70,10198491,True,True,None,None,None,None
8,10198491,G7B04,37.60,KG,GITPD,74152024.0,1879602,3202656509,NaN,NaN,...,120.0,80.0,4.70,10198491,True,True,None,None,None,None
9,10198491,G7B04,37.60,KG,GITPD,74152024.0,1879603,3202656509,NaN,NaN,...,120.0,80.0,4.70,10198491,True,True,None,None,None,None


In [82]:
df['Teilbar_durch'] =  df['MM_TH_Pro_Palette'].astype(int) / df['Case_on_Layer'].astype(int)
# #drop MM_Case_per_PAL and Case_on_Layer
df['Teilbar_durch'] = df['Teilbar_durch'].astype(float)
# # df = df.drop(columns=['MM_Case_per_PAL', 'Case_on_Layer'])
# # set unique ID for each row 
df['Pal_ID'] = [uuid.uuid4() for _ in range(len(df.index))]
df['Teilbar_durch'] = df['Teilbar_durch'].apply(lambda x: round(x))
df['Teilhöhe'] = df['height_PAL'].astype(float) / df['Teilbar_durch']
df['Teilhöhe'] = df['Teilhöhe'].apply(lambda x: round(x, 2)) + 1
return df 